In [ ]:
# imports

# standard import of pandas
import pandas as pd

In [ ]:
# getting dataframes

%store -r start_orders
%store -r start_order_process
%store -r start_intern_study
%store -r start_campaign

In [ ]:
clean_orders = start_orders.copy()
clean_order_process = start_order_process.copy()
clean_intern_study = start_intern_study.copy()
clean_campaign = start_campaign.copy()

In [ ]:
# function to make all the columns lower and snake case

def columns_format(data_frame):
	data_frame.columns = data_frame.columns.str.lower()
	data_frame.columns = data_frame.columns.str.replace(" ", "_")


In [ ]:
columns_format(clean_orders)
columns_format(clean_order_process)
columns_format(clean_intern_study)
columns_format(clean_campaign)

In [ ]:
# dropping columns in orders.csv

clean_orders.drop(["index","customer_id", "customer_name", "origin_channel","country/region", "city", "postal_code", "category", "sub-category", "product_id", "sales", "quantity", "discount", "profit"], axis=1, inplace=True)
clean_orders.drop_duplicates(inplace=True)
clean_orders.head(2)
clean_orders.duplicated().value_counts()

In [ ]:
# dropping row_id

clean_order_process.drop("row_id", axis=1, inplace=True)
clean_order_process.drop_duplicates(inplace=True)
clean_order_process.head()
clean_order_process.duplicated().value_counts()

In [ ]:
# dropping duplicates in intern data

clean_intern_study.drop_duplicates(inplace=True)
clean_intern_study.head(2)
clean_intern_study.duplicated().value_counts()

In [ ]:
# dropping columns in campaign data

clean_campaign.drop(["customer_name"], axis=1, inplace=True)
clean_campaign.head(2)

In [ ]:
first_merge = clean_orders.merge(clean_order_process, how="outer", on="order_id")

first_merge

In [ ]:
second_merge = first_merge.merge(clean_intern_study, how="outer", on="order_id")

second_merge

In [ ]:
final_merge = second_merge.merge(clean_campaign, how="outer", on="order_id")
final_merge

In [ ]:
final_merge[final_merge['ship_mode_y'].isnull()]

In [ ]:
## fill null values from ship_mode:y with values from ship_mode_x
final_merge['ship_mode_y'] = final_merge['ship_mode_y'].fillna(final_merge['ship_mode_x'])

## check there are no null values in ship_mode_y
final_merge[final_merge['ship_mode_y'].isnull()]

final_merge.drop("ship_mode_x", axis=1, inplace=True)
final_merge.rename(columns={"ship_mode_y": "ship_mode"}, inplace=True)

final_merge['ship_mode'].replace({'Standard Class': "Standard Processing"}, inplace=True)
final_merge['ship_mode'].replace({'Second Class': "Standard Processing"}, inplace=True)
final_merge['ship_mode'].replace({'First Class': "Express"}, inplace=True)
final_merge.head()

In [ ]:
final_merge.insert(2, "order_date", final_merge["order_date_x"])
final_merge.drop(["order_date_x", "order_date_y"], axis=1, inplace=True)
final_merge.head(15)

In [ ]:
#final_merge.insert(2, "ship_mode", final_merge["ship_mode_y"])
#final_merge.drop(["ship_mode_x", "ship_mode_y"], axis=1, inplace=True)
final_merge.info()

In [ ]:
final_merge['order_date'] = pd.to_datetime(final_merge["order_date"], format="%d/%m/%Y")
final_merge['ready_to_ship_date'] = pd.to_datetime(final_merge["ready_to_ship_date"], format="%d/%m/%Y")
final_merge['pickup_date'] = pd.to_datetime(final_merge["pickup_date"], format="%d/%m/%Y")
final_merge['arrival_scan_date'] = pd.to_datetime(final_merge["arrival_scan_date"], format="%d/%m/%Y")
final_merge['on_truck_scan_date'] = pd.to_datetime(final_merge["on_truck_scan_date"], format="%d/%m/%Y")

In [ ]:
final_merge.info()

In [ ]:
## fill null values from ship_mode:y with values from ship_mode_x
final_merge['pickup_date'] = final_merge['pickup_date'].fillna(final_merge['on_truck_scan_date'])
final_merge.drop("on_truck_scan_date", axis=1, inplace=True)
final_merge[final_merge['pickup_date'].isnull()]


In [ ]:
%store final_merge